In [ ]:
"""Test Kserve model inference service for MNIST digits"""

from kserve import KServeClient
import numpy as np
import requests, json
import matplotlib.pyplot as plt
from tensorflow import keras
import random

KServe = KServeClient()

# get inferenceservice url
isvc_resp = KServe.get("digits-recognizer", namespace="kubeflow-user")
isvc_url = isvc_resp['status']['address']['url']

#load the test dataset
(_, _), (x_test, y_test) = keras.datasets.mnist.load_data()

#generate a random number
random_number = random.randint(0, 100)

# test the model via rest api
print("\n\n<==== Input ====>")
print(f"Number: {y_test[random_number]}")
#show the input number
plt.imshow(x_test[random_number])
plt.show()

t = np.array(x_test[random_number]).reshape(-1,28,28,1)

inference_input = {'instances': t.tolist()}
response = requests.post(isvc_url, json=inference_input)
r = json.loads(response.text)

print("\n\n<==== Output ====>")
print("Prediction: {}".format(np.argmax(r["predictions"])))

In [ ]:
"""Test Kserve model inference service for MNIST digits"""

from kserve import KServeClient
import numpy as np
import requests, json
import pandas as pd
from tensorflow import keras

KServe = KServeClient()

# get inferenceservice url
isvc_resp = KServe.get("digits-recognizer", namespace="kubeflow-user")
isvc_url = isvc_resp['status']['address']['url']

# load the test dataset
(_, _), (x_test, y_test) = keras.datasets.mnist.load_data()

# create empty lists to store data for the table
input_data = []
predicted_data = []
accuracies = []

# iterate through test dataset and make API calls for inference
for i in range(len(x_test)):
    t = np.array(x_test[i]).reshape(-1,28,28,1)

    inference_input = {'instances': t.tolist()}
    response = requests.post(isvc_url, json=inference_input)
    r = json.loads(response.text)

    input_data.append(y_test[i])
    predicted_data.append(np.argmax(r["predictions"]))
    accuracies.append(1 if y_test[i] == np.argmax(r["predictions"]) else 0)

# create a Pandas DataFrame to store the data
data = {
    'Input': input_data,
    'Predicted': predicted_data,
    'Accuracy': accuracies
}
df = pd.DataFrame(data)

# calculate overall accuracy
overall_accuracy = df['Accuracy'].mean() * 100

# display the comparison table and overall accuracy
print("\n<==== Comparison Table ====>")
print(df)
print(f"\nOverall Accuracy: {overall_accuracy:.2f}%")
